# Lossy compression to prevent evasion and poisoning

[Slides](https://danjacobellis.github.io/FTML/compress.slides.html)

<script>
    document.querySelector('head').innerHTML += '<style>.slides { zoom: 1.75 !important; }</style>';
</script>

<center> <h1>
Lossy compression to prevent evasion and poisoning
</h1> </center>

<center> <h3>
Dan Jacobellis, Matthew Qin
</h3> </center>

## Evasion
* Exploit knowledge a model that's already been trained
* Example: email spam filter
  * Attacker wants to avoid detection while preserving the semantic content an email
  * Full or partial knowledge of model can be used to find "magic words" that cause an email to be classified as not spam

## Poisoning
* Attacker contaminates dataset, usually with the goal of introducing a backdoor
* Example: facial recognition
  * Attacker wants to prevent facial recognition from working on one or more subjects
  * Attacker uploads altered image to public where dataset is sourced for training

## Fast gradient sign method
$$x_{\text{adv}} = x + \epsilon*\text{sign}(\nabla_xJ(\theta, x, y))$$

<p style="text-align:center;">
<img src="_images/doog.png" width=700 height=700 class="center">
</p>

![](img/doog.png)

## "Robust" features
* "Overall, attaining models that are robust and interpretable will require explicitly encoding human priors into the training process"

<p style="text-align:center;">
<img src="_images/robust_doog.png" width=700 height=700 class="center">
</p>

![](img/robust_doog.png)

## Lossy compression

<p style="text-align:center;">
<img src="_images/lossy_lossless.png" width=800 height=800 class="center">
</p>

![](img/lossy_lossless.png)

## Lossy compression to prevent evasion

<p style="text-align:center;">
<img src="_images/JPEG_evasion1.png" width=800 height=800 class="center">
</p>

![](img/JPEG_evasion1.png)

## Lossy compression to prevent evasion

<p style="text-align:center;">
<img src="_images/JPEG_evasion2.png" width=800 height=800 class="center">
</p>

![](img/JPEG_evasion2.png)

## Visual perception of quantization

<p style="text-align:center;">
<img src="_images/dither.png" width=700 height=700 class="center">
</p>

![](img/dither.png)

* Most perturbations are imperceptible if contained in the four least significant bits.

## Defending against larger perturbations

<p style="text-align:center;">
<img src="_images/jpeg_vs_gif.png" width=700 height=700 class="center">
</p>

![](img/jpeg_vs_gif.png)

https://deepai.org/publication/frequency-tuned-universal-adversarial-attacks#S2.SS3.p1

## How lossy to compress?

$$ \text{Performance on perturbed test sample} = \begin{cases} \text{label confidence} & \text{label is correct} \\ -\text{label confidence} & \text{label is incorrect} \end{cases}$$

<p style="text-align:center;">
<img src="_images/accuracy_vs_fidelity.png" width=700 height=700 class="center">
</p>

![](img/accuracy_vs_fidelity.png)

## Gradient matching

* Proposed in 2020 as a more efficient method to poison large datasets
* Attacker chooses a specific image and chooses a label that they want the image to be classified as
* Using small perturbations to as little as 0.1% of the dataset, the chosen image can be classified as desired by the attacker
* Requires larger perturbations that evasion attacks but are still mostly imperceptible
* Does not require full knowledge of model architecture. Shown to translate to different models
* Example: Poisoning data by assuming a resnet20 model still works when a VGG13 model is trained  

## Sanitization to prevent poisoning

<p style="text-align:center;">
<img src="_images/CIFAR_compare.png" width=650 height=650 class="center">
</p>

![](img/CIFAR_compare.png)

## Compressed training to prevent poisoning

* An attacker wants the poisoning attack to be imperceptible
* Requires the "poison" part of the data to be mostly contained in the least significant bits or high frequencies
* Instead of training in pixel space, train on quantized transform coefficients
  * Discards details in the least significant bits as well as high frequencies
  * Details important for classification are kept

## Audio classification: baseline
* Dataset: Speech commands
  * One second speech segments of 8 possible words
  * 'stop,' 'down,' 'no,' 'right,' 'go,' 'up,' 'yes,' 'left'
* Baseline model:
  * Input size: $128 \times 128$ time-frequency distribution represented at full precision
  * 119.52 MiB Feature size
  * 2.26 GFLOPs per pass
  * Achieves test accuracy of about 84% 

## Audio classification: VQ + BNN
* Encode 2x2 time-frequency blocks via vector quantization
  * Use mini-batch k-means to learn codebook of 16 vectors (4 bits)
  * Compression ratio of 16:1 (before any entropy coding)
* Input size: $64 \times 64 \times 4$ binary codes

| <audio controls="controls"><source src="./_static/left01.wav" type="audio/wav"></audio>    | <audio controls="controls"><source src="./_static/right01.wav" type="audio/wav"></audio>    | <audio controls="controls"><source src="./_static/yes01.wav" type="audio/wav"></audio>    | <audio controls="controls"><source src="./_static/no01.wav" type="audio/wav"></audio>    |
|--------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------|
| <audio controls="controls"><source src="./_static/left01_vq.wav" type="audio/wav"></audio> | <audio controls="controls"><source src="./_static/right01_vq.wav" type="audio/wav"></audio> | <audio controls="controls"><source src="./_static/yes01_vq.wav" type="audio/wav"></audio> | <audio controls="controls"><source src="./_static/no01_vq.wav" type="audio/wav"></audio> |

## Audio classification: VQ + BNN
* Input size: $64 \times 64 \times 4$ binary codes
* 3.74 MiB feature size 
* Multiply-accumulate instead of FP
  * 4-way MAC unit uses about 55% of the area of a FP16 FPU
  * $4-8 \times$ more power efficient compared to bfloat16
  * $>20 \times$ more power efficient compared to FP32
* Achieves test accuracy of about 79% (down from baseline of 84%)